## Create Virtual Environment

It is a best practice to create and activate a new Virtual environment when working with lab samples.  This helps to not overwrite the global settings of your main Python install or overwrite packages across your projects.

NOTE: These notebooks were tested with Python 3.11.

## Install Libraries

Install the required libraries and SDKs.

In [ ]:
!pip install langchain 
!pip install openai 
!pip install python-environ 
!pip install psycopg2 
!pip install pandas
!pip install langchain
!pip install langchain_experimental
!pip install openai
!pip install tiktoken

Import the modules.

In [ ]:
import psycopg2

Create a connection string and test the connection. Be sure to replace the `SUFFIX`.

In [ ]:
username = 'wsuser'
password = 'Solliance123'
host = 'pgsqldevSUFFIXflex14.postgres.database.azure.com'
port = '5432'
dbname = 'ailabs'

connection_string = f'postgresql+psycopg2://{username}:{password}@{host}:{port}/{dbname}'

# Establish a connection to the PostgreSQL database
conn = psycopg2.connect(
    host=host,
    port=5432,
    user=username,
    password=password,
    database=dbname
)

Use sqlalchemy to create a db engine.

In [ ]:
#load the csv data to the database
import pandas as pd
from sqlalchemy import create_engine

db_engine = create_engine(connection_string)

Use pandas to load CSV files and then import them into the database.

In [ ]:
df = pd.read_csv('./data/calendar.csv')
df.to_sql('calendar', db_engine, if_exists='replace', index=False)

df = pd.read_csv('./data/listings.csv')
df.to_sql('listings', db_engine, if_exists='replace', index=False)

df = pd.read_csv('./data/reviews.csv')
df.to_sql('reviews', db_engine, if_exists='replace', index=False)

## Azure OpenAI Studio

## GPT Text Embeddings

Now that you have deployed a simple model via Azure Machine Learning Studio, let's look at redo-ing our embeddings using a different embedding model called `text-embedding-ada-002`.

This model is not deployed via Azure Machine Learning Studio, but is a part of Azure OpenAI.

- Open the Azure Portal
- Search for **Azure Open AI**
- Under **Resource Management**, select **Keys and Endpoint**
- Record the endpoing and the key
- Under **Resource Management**, select **Model deployments**
- Select **Manage Delopments**
- Select **Create new deployment**
- Select the **gpt4** model
- For the deployment name, type **completions4**
- Select **Create**
- Once the model is deployed, run the following cells to create a new Chat agent. Be sure to replace the endpoint and key with the ones you just recorded.

In [ ]:
from langchain.chat_models import AzureChatOpenAI

azure_endpoint = "YOUR_ENDPOINT"
azure_key = 'YOUR_KEY'

llm=AzureChatOpenAI(
    temperature=0,
    openai_api_key =azure_key,
    openai_api_base = azure_endpoint,
    openai_api_version="2023-03-15-preview",
    openai_api_type="azure",
    model_name="gpt4",
    deployment_name="completions4"
)

In [ ]:
def handle_parsing_errors():
    pass

Setup a SQL agent with a SQLDatabaseToolkit.

In [ ]:
from langchain_experimental.sql.base import SQLDatabaseChain
#from langchain_experimental.sql.base.sql_database import SQLDatabase
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.agents import create_sql_agent
from langchain.callbacks import get_openai_callback

from langchain import SQLDatabase

db = SQLDatabase.from_uri(connection_string)

toolkit = SQLDatabaseToolkit(db=db, llm=llm)

agent_executor = create_sql_agent(
    llm=llm,
    toolkit=toolkit,
    verbose=True,
    handle_parsing_errors=handle_parsing_errors
)

Execute a query against the agent.

In [ ]:
# Create db chain
QUERY = """
Given an input question, first create a syntactically correct postgresql query to run, then look at the results of the query and return the answer.
Use the following format:

"Question": "Question here"
"SQLQuery": "SQL Query to run"
"SQLResult": "Result of the SQLQuery"
"Answer": "Final answer here"

"{question}"
"""

question_text = "What are the details of the rental that is closest to the Space Needle?"

question = QUERY.format(question=question_text)

with get_openai_callback() as cb:
    res = agent_executor.run(question)

Show the result.

In [ ]:
print(res)